<a href="https://colab.research.google.com/github/michael-vdr/Twittorials/blob/master/Quick_win_wordcount_meta_title_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>





### Resources
You can find example CSV files in the sheet: https://docs.google.com/spreadsheets/d/18e5Xd1l2dHojd91L7e1saY0xGufsIdhPUYe8yDL88Ag/edit?usp=sharing


### Step 0: Language settings

In [1]:
langcode="en" #@param {type:"string"}
language="english" #@param {type:"string"}

### Step 1 : Import modules

In [2]:
#import modules
import pandas as pd 
#input/output modules for uploading and downloading files
from google.colab import files
import io
#install nlp modules not yet available in colab
!pip install stop_words
!pip install python-levenshtein
import re #NLP regex module
import nltk #general NLP module
from nltk.stem.snowball import SnowballStemmer #NLP stemming module
from Levenshtein import * #NLP distance caluclator
from stop_words import get_stop_words #stop words NLP library
nltk.download('punkt') #tokenizer NLP 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Step 2 : Upload the page title file

In [3]:
#Step 2: upload the page title file
uploaded = files.upload()
filename = str(uploaded).split("'")[1]
dftitles = pd.read_csv(io.BytesIO(uploaded[filename]))
dftitles.columns=["Page","Title"]
dftitles.head()

Saving Sheets - Pages and titles (1).csv to Sheets - Pages and titles (1) (1).csv


,Page,Title
0,https://www.toys.com/toddlers,Games for toddlers
1,https://www.toys.com/stress-relief-toy,Toys for stress relief
2,https://www.toys.com/unicorn-toy-figure,Unicorn toy figures
3,https://www.toys.com/pvc-adjustable-basketstand,A PVC adjustable basketstand


### Step 3: Upload second CSV file: the search queries by page 

In [4]:
uploaded = files.upload()
filename = str(uploaded).split("'")[1]
dfqueries = pd.read_csv(io.BytesIO(uploaded[filename]))
dfqueries=dfqueries.iloc[:, : 2]
dfqueries.columns=["Query","Page"]
dfqueries.head()

Saving Sheets - Queries and pages.csv to Sheets - Queries and pages (6).csv


,Query,Page
0,toy for a 2 year old,https://www.toys.com/toddlers
1,toy for a 3 year old,https://www.toys.com/toddlers
2,anxiety relief toy,https://www.toys.com/stress-relief-toy
3,toy for adhd kid,https://www.toys.com/stress-relief-toy
4,unicorn toy figure,https://www.toys.com/unicorn-toy-figure


### Step 4: Combining dataframes

In [5]:
df = pd.merge(dfqueries,dftitles,how="inner",on='Page')
df.head()

,Query,Page,Title
0,toy for a 2 year old,https://www.toys.com/toddlers,Games for toddlers
1,toy for a 3 year old,https://www.toys.com/toddlers,Games for toddlers
2,anxiety relief toy,https://www.toys.com/stress-relief-toy,Toys for stress relief
3,toy for adhd kid,https://www.toys.com/stress-relief-toy,Toys for stress relief
4,unicorn toy figure,https://www.toys.com/unicorn-toy-figure,Unicorn toy figures


### Step 5: Count number of words

In [6]:
stop_words = get_stop_words(langcode)

def number_words(sentence):
  sentencewords = nltk.word_tokenize(str(sentence).lower()) 
  not_stop_words=[]
  for word in sentencewords:
    if(word not in stop_words):
        not_stop_words.append(word)
  sentencelength = len(not_stop_words)
  return sentencelength

df["Words in query"]=df.apply(lambda x:number_words(x["Query"]),axis=1)
df["Words in title"]=df.apply(lambda x:number_words(x["Title"]),axis=1)
df.head()

,Query,Page,Title,Words in query,Words in title
0,toy for a 2 year old,https://www.toys.com/toddlers,Games for toddlers,4,2
1,toy for a 3 year old,https://www.toys.com/toddlers,Games for toddlers,4,2
2,anxiety relief toy,https://www.toys.com/stress-relief-toy,Toys for stress relief,3,3
3,toy for adhd kid,https://www.toys.com/stress-relief-toy,Toys for stress relief,3,3
4,unicorn toy figure,https://www.toys.com/unicorn-toy-figure,Unicorn toy figures,3,3


### Step 6: Count number of common words
words of query in title

In [12]:
def common_words(query,title):
    stemmer  = SnowballStemmer(language, ignore_stopwords=True)
    titlewords = nltk.word_tokenize(str(title).lower())
    stemtitlewords = [stemmer.stem(word) for word in titlewords]
    querywords = nltk.word_tokenize(str(query).lower()) 
    stemquerywords = [stemmer.stem(word) for word in querywords]
    titlenospaces=title.replace(' ','').lower()
    title_only_text = re.sub(r'\W+', '', titlenospaces)
    
    lev_distance=0  
    commonwords=0

    for word in stemquerywords:
      if(word not in stop_words):
          #literal word check
          if(word in title_only_text):
              commonwords=commonwords+1         
          else:
          #check with word with title stemmed and for misspellings
            for word2 in stemtitlewords:
                lev_distance = distance(str(word),str(word2))
                procent_distance = lev_distance/(len(word)+len(word2)/2)
                if procent_distance<0.15:
                    commonwords=commonwords+1
    return commonwords
    
df["Common words"]=df.apply(lambda x:common_words(x["Query"],x["Title"]),axis=1)
df.sample(5)

,Query,Page,Title,Words in query,Words in title,Common words
5,unicorm toy figure,https://www.toys.com/unicorn-toy-figure,Unicorn toy figures,3,3,3
7,basketball hoop for kids,https://www.toys.com/pvc-adjustable-basketstand,A PVC adjustable basketstand,3,3,0
2,anxiety relief toy,https://www.toys.com/stress-relief-toy,Toys for stress relief,3,3,2
8,buy basketball ring 5 feet,https://www.toys.com/pvc-adjustable-basketstand,A PVC adjustable basketstand,5,3,0
4,unicorn toy figure,https://www.toys.com/unicorn-toy-figure,Unicorn toy figures,3,3,3


### Step 7 sort and explore table

In [9]:
print("Possible optimizations")
print("----------------------")

df=df.sort_values(by=['Common words', 'Words in query'], ascending=[True, False])
df.head()

Possible optimizations
----------------------


,Query,Page,Title,Words in query,Words in title,Common words
8,buy basketball ring 5 feet,https://www.toys.com/pvc-adjustable-basketstand,A PVC adjustable basketstand,5,3,0
0,toy for a 2 year old,https://www.toys.com/toddlers,Games for toddlers,4,2,0
1,toy for a 3 year old,https://www.toys.com/toddlers,Games for toddlers,4,2,0
7,basketball hoop for kids,https://www.toys.com/pvc-adjustable-basketstand,A PVC adjustable basketstand,3,3,0
6,pony game,https://www.toys.com/unicorn-toy-figure,Unicorn toy figures,2,3,0


### Additional step: Export and download

In [ ]:
#automatically download
df.to_csv('export.csv')
files.download('export.csv') 